- 주어진 가상의 고객행동/구매 data를 가지고 사전 행동을 분석하고 EDA 결과 및 행동 특성에 따라 유형화(고객 sgmentation) 및 RFM Class Score 부여

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
filename1 = '/content/drive/MyDrive/현대자동차2022/DM1_DS_99_HCKRTHN_CNTT_F.csv'
filename2 = '/content/drive/MyDrive/현대자동차2022/DM1_DS_99_HCKRTHN_DEMO_F.csv'
filename3 = '/content/drive/MyDrive/현대자동차2022/DM1_DS_99_HCKRTHN_PROS_ITG_F.csv'

Mounted at /content/drive


In [ ]:
#차량 정보
carinfo_df = pd.read_csv(filename1,encoding='cp949')
#고객 접촉 이력 정보
cus = pd.read_csv(filename2,encoding='cp949')
#채널 인입 이력
inc = pd.read_csv(filename3,encoding='cp949')

##데이터 전처리

---
1. 해당되는 고객의 차량 계약 이력 및 정보
2. 1:n 관계로 다수의 차량 구매 이력


In [ ]:
#차량 정보
carinfo_df = pd.read_csv(filename1,encoding='cp949')
carinfo_df.head(5)

,CAR_ID,CNTT_DT,VEHL_CD,DOM_MDL_NM,PAC_SCN_NM,SALE_CGRD_NM,SALE_TRIM_NM,SALE_ENG_NM,CUS_ID
0,C98N3HQIS4G67,20210302,IG,그랜저IG 자가용 하이브리드 PREMIUM A/T F/L,승용,대형,PREMIUM,하이브리드,APW9ZIWDQQNYY8LK
1,C18MB8E9FEO8O,20220117,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X
2,CXMMZDI2JRWCX,20200511,DN,쏘나타DN8 20MY 자가용 가솔린 2.0 CVVL 프리미엄,승용,중형,프리미엄 패밀리,가솔린 2.0 CVVL,A4W2GG6ZLMZ81DB3
3,C934SRCKF4H44,20220104,NX,The all-new 투싼 하이브리드 인스퍼레이션 2WD A,RV,준중형SUV,인스퍼레이션,하이브리드,ATHNS3F4766FKDW3
4,CU1BBNUEHOTDM,20220210,JJ,The All new G80 가솔린 2.5T AWD 19인치,승용,대형,19인치A SDS2,가솔린 2.5T,A8D7NHFKA3K83QH4


In [ ]:
carinfo_df.rename(columns = {
    "CAR_ID":"차량ID",
    "CNTT_DT":"계약일자",
    "VEHL_CD":"차종코드",
    "DOM_MDL_NM":"모델명",
    "PAC_SCN_NM":"승상구분명",
    "SALE_CGRD_NM":"판매차급명",
    "SALE_TRIM_NM":"판매트림명",
    "SALE_ENG_NM":"판매엔진명",
    "CUS_ID":"고객ID",
}, inplace = True)
carinfo_df

,차량ID,계약일자,차종코드,모델명,승상구분명,판매차급명,판매트림명,판매엔진명,고객ID
0,C98N3HQIS4G67,20210302,IG,그랜저IG 자가용 하이브리드 PREMIUM A/T F/L,승용,대형,PREMIUM,하이브리드,APW9ZIWDQQNYY8LK
1,C18MB8E9FEO8O,20220117,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X
2,CXMMZDI2JRWCX,20200511,DN,쏘나타DN8 20MY 자가용 가솔린 2.0 CVVL 프리미엄,승용,중형,프리미엄 패밀리,가솔린 2.0 CVVL,A4W2GG6ZLMZ81DB3
3,C934SRCKF4H44,20220104,NX,The all-new 투싼 하이브리드 인스퍼레이션 2WD A,RV,준중형SUV,인스퍼레이션,하이브리드,ATHNS3F4766FKDW3
4,CU1BBNUEHOTDM,20220210,JJ,The All new G80 가솔린 2.5T AWD 19인치,승용,대형,19인치A SDS2,가솔린 2.5T,A8D7NHFKA3K83QH4
...,...,...,...,...,...,...,...,...,...
72780,CF5XYYWGM4962,20111025,MD,아반떼MD 자가용 가솔린 1.6 A 오토 11MY,승용,준중형,A,가솔린 1.6,ACWSOYPRRWLHL2I1
72781,C91VRA1RAE6EL,20160504,TL,투싼TL 디젤 1.7 Premium 2WD DCT,RV,준중형SUV,Premium,디젤 1.7,ACWSOYPRRWLHL2I1
72782,COAFAA0KQ2F1N,20220203,NX,The all-new 투싼 하이브리드 프리미엄 2WD A/T,RV,준중형SUV,프리미엄,하이브리드,AF3D2H4N6PO5PY24
72783,COBZ4YAULMIMD,20100729,JA,G스타렉스 디젤 12인승 웨건 자가용 CVX 오토 11MY,소형상용,소형버스,CVX,디젤,APXWMUKE36MD8PX1


In [ ]:
##cus df 열 변경
cus.rename(columns = {
    "CUS_ID":"고객ID",
    "PSN_BIZR_YN":"개인사업자여부",
    "TYMD":"생년",
    "SEX_SCN_NM":"성별구분명",
}, inplace = True)
cus

,고객ID,개인사업자여부,생년,성별구분명
0,AMR9MOP756HN28V9,N,1990.0,여자
1,AL2E2HLDMN50FN9E,N,1955.0,남자
2,APW9ZIWDQQNYY8LK,N,1992.0,남자
3,AGBCIS57DPHRRP9X,N,1977.0,남자
4,A4W2GG6ZLMZ81DB3,N,1974.0,여자
...,...,...,...,...
89575,AF3D2H4N6PO5PY24,N,1968.0,남자
89576,AQSF1UNGCVX5CVJH,N,1984.0,남자
89577,ARJA0WA3EEXVZQFW,N,1982.0,남자
89578,APXWMUKE36MD8PX1,N,1984.0,남자


In [ ]:
##inc df 열 변경
inc.rename(columns = {
    "CUS_ID":"고객ID",
    "CASE_ID":"접촉ID",
    "INP_DTM":"인입일시",
    "CHAN_SCN_NM":"채널구분명",
    "CHAN_DTL_SCN_NM":"채널상세구분명",
    "IRD_VEHL_CD":"관심차량",
}, inplace = True)
inc

,고객ID,접촉ID,인입일시,채널구분명,채널상세구분명,관심차량
0,AKZWMZWJ3HTYTOSS,BK69SK0W9NOEXIQU,2022-01-09 16:00:00,시승시스템,시승,JW
1,AAMN599HM0XFRIRK,BCPT4WL1QHD8LK1M,2022-01-12 10:00:00,시승시스템,시승,QX
2,AP8ERGN3MGXWD0W1,BUYAFEX9M7PATNIF,2022-01-09 18:41:28,현대닷컴,견적,NX
3,ASIYQ5LM878U45P5,BVCR9MK9FIRXEUCL,2022-01-11 18:41:21,이벤트시스템,구매상담신청,NaN
4,A8KWVKZPITWG0PCO,BNQ8X4SUSC120MRF,2022-01-10 09:03:09,판매기회_상담완료,지속관리,JJ
...,...,...,...,...,...,...
159166,AS7OEWKWHIAK2WZ1,BS94L650O9PWIXE1,2022-03-07 18:09:35,판매기회_상담완료,지속관리,NX
159167,AVFD9EAOXP2BYXQY,BIL593DS4LTUT2JP,2022-02-03 10:23:16,판매기회_상담완료,지속관리,NaN
159168,AA9FVJGB2L5OG3IW,B26O6YSJL6XOIHIZ,2022-03-14 11:05:38,판매기회_상담완료,지속관리,LC
159169,AIJ2KUXDXEV0VDUS,BXQDUBABH92L3XOQ,2022-05-09 10:04:17,판매기회_상담완료,지속관리,NaN


In [ ]:
inc_1 = inc[['고객ID','채널구분명','관심차량']]

In [ ]:
carinfo_df.isna().any()

차량ID     False
계약일자     False
차종코드     False
모델명      False
승상구분명    False
판매차급명    False
판매트림명     True
판매엔진명     True
고객ID     False
dtype: bool

In [ ]:
carinfo_df.isnull().sum()

차량ID      0
계약일자      0
차종코드      0
모델명       0
승상구분명     0
판매차급명     0
판매트림명    11
판매엔진명    73
고객ID      0
dtype: int64

In [ ]:
#데이터의 고유치 갯수
carinfo_df.nunique()

차량ID     72785
계약일자      3713
차종코드        59
모델명       2515
승상구분명        3
판매차급명       12
판매트림명      326
판매엔진명       79
고객ID     51397
dtype: int64

In [ ]:
carinfo_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 72785 entries, 0 to 72784
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   차량ID    72785 non-null  object
 1   계약일자    72785 non-null  int64 
 2   차종코드    72785 non-null  object
 3   모델명     72785 non-null  object
 4   승상구분명   72785 non-null  object
 5   판매차급명   72785 non-null  object
 6   판매트림명   72774 non-null  object
 7   판매엔진명   72712 non-null  object
 8   고객ID    72785 non-null  object
dtypes: int64(1), object(8)
memory usage: 5.0+ MB


In [ ]:
import datetime
carinfo_df['계약일자'] = carinfo_df['계약일자'].astype('str')
carinfo_df['계약일자'] = carinfo_df['계약일자'].apply(lambda _ : datetime.datetime.strptime(_,'%Y%m%d'))

In [ ]:
carinfo_df

,차량ID,계약일자,차종코드,모델명,승상구분명,판매차급명,판매트림명,판매엔진명,고객ID
0,C98N3HQIS4G67,2021-03-02,IG,그랜저IG 자가용 하이브리드 PREMIUM A/T F/L,승용,대형,PREMIUM,하이브리드,APW9ZIWDQQNYY8LK
1,C18MB8E9FEO8O,2022-01-17,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X
2,CXMMZDI2JRWCX,2020-05-11,DN,쏘나타DN8 20MY 자가용 가솔린 2.0 CVVL 프리미엄,승용,중형,프리미엄 패밀리,가솔린 2.0 CVVL,A4W2GG6ZLMZ81DB3
3,C934SRCKF4H44,2022-01-04,NX,The all-new 투싼 하이브리드 인스퍼레이션 2WD A,RV,준중형SUV,인스퍼레이션,하이브리드,ATHNS3F4766FKDW3
4,CU1BBNUEHOTDM,2022-02-10,JJ,The All new G80 가솔린 2.5T AWD 19인치,승용,대형,19인치A SDS2,가솔린 2.5T,A8D7NHFKA3K83QH4
...,...,...,...,...,...,...,...,...,...
72780,CF5XYYWGM4962,2011-10-25,MD,아반떼MD 자가용 가솔린 1.6 A 오토 11MY,승용,준중형,A,가솔린 1.6,ACWSOYPRRWLHL2I1
72781,C91VRA1RAE6EL,2016-05-04,TL,투싼TL 디젤 1.7 Premium 2WD DCT,RV,준중형SUV,Premium,디젤 1.7,ACWSOYPRRWLHL2I1
72782,COAFAA0KQ2F1N,2022-02-03,NX,The all-new 투싼 하이브리드 프리미엄 2WD A/T,RV,준중형SUV,프리미엄,하이브리드,AF3D2H4N6PO5PY24
72783,COBZ4YAULMIMD,2010-07-29,JA,G스타렉스 디젤 12인승 웨건 자가용 CVX 오토 11MY,소형상용,소형버스,CVX,디젤,APXWMUKE36MD8PX1


In [ ]:
new_df = pd.merge(carinfo_df, cus, left_on='고객ID',right_on='고객ID')
new_df

,차량ID,계약일자,차종코드,모델명,승상구분명,판매차급명,판매트림명,판매엔진명,고객ID,개인사업자여부,생년,성별구분명
0,C98N3HQIS4G67,2021-03-02,IG,그랜저IG 자가용 하이브리드 PREMIUM A/T F/L,승용,대형,PREMIUM,하이브리드,APW9ZIWDQQNYY8LK,N,1992.0,남자
1,C18MB8E9FEO8O,2022-01-17,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X,N,1977.0,남자
2,CXMMZDI2JRWCX,2020-05-11,DN,쏘나타DN8 20MY 자가용 가솔린 2.0 CVVL 프리미엄,승용,중형,프리미엄 패밀리,가솔린 2.0 CVVL,A4W2GG6ZLMZ81DB3,N,1974.0,여자
3,C934SRCKF4H44,2022-01-04,NX,The all-new 투싼 하이브리드 인스퍼레이션 2WD A,RV,준중형SUV,인스퍼레이션,하이브리드,ATHNS3F4766FKDW3,N,1981.0,여자
4,CU1BBNUEHOTDM,2022-02-10,JJ,The All new G80 가솔린 2.5T AWD 19인치,승용,대형,19인치A SDS2,가솔린 2.5T,A8D7NHFKA3K83QH4,N,1965.0,여자
...,...,...,...,...,...,...,...,...,...,...,...,...
71056,CF5XYYWGM4962,2011-10-25,MD,아반떼MD 자가용 가솔린 1.6 A 오토 11MY,승용,준중형,A,가솔린 1.6,ACWSOYPRRWLHL2I1,N,1979.0,남자
71057,C91VRA1RAE6EL,2016-05-04,TL,투싼TL 디젤 1.7 Premium 2WD DCT,RV,준중형SUV,Premium,디젤 1.7,ACWSOYPRRWLHL2I1,N,1979.0,남자
71058,COAFAA0KQ2F1N,2022-02-03,NX,The all-new 투싼 하이브리드 프리미엄 2WD A/T,RV,준중형SUV,프리미엄,하이브리드,AF3D2H4N6PO5PY24,N,1968.0,남자
71059,COBZ4YAULMIMD,2010-07-29,JA,G스타렉스 디젤 12인승 웨건 자가용 CVX 오토 11MY,소형상용,소형버스,CVX,디젤,APXWMUKE36MD8PX1,N,1984.0,남자


In [ ]:
inc_1

,고객ID,채널구분명,관심차량
0,AKZWMZWJ3HTYTOSS,시승시스템,JW
1,AAMN599HM0XFRIRK,시승시스템,QX
2,AP8ERGN3MGXWD0W1,현대닷컴,NX
3,ASIYQ5LM878U45P5,이벤트시스템,NaN
4,A8KWVKZPITWG0PCO,판매기회_상담완료,JJ
...,...,...,...
159166,AS7OEWKWHIAK2WZ1,판매기회_상담완료,NX
159167,AVFD9EAOXP2BYXQY,판매기회_상담완료,NaN
159168,AA9FVJGB2L5OG3IW,판매기회_상담완료,LC
159169,AIJ2KUXDXEV0VDUS,판매기회_상담완료,NaN


In [ ]:
df = pd.merge(new_df, inc_1 , left_on='고객ID', right_on='고객ID')
df

,차량ID,계약일자,차종코드,모델명,승상구분명,판매차급명,판매트림명,판매엔진명,고객ID,개인사업자여부,생년,성별구분명,채널구분명,관심차량
0,C98N3HQIS4G67,2021-03-02,IG,그랜저IG 자가용 하이브리드 PREMIUM A/T F/L,승용,대형,PREMIUM,하이브리드,APW9ZIWDQQNYY8LK,N,1992.0,남자,시승시스템,IG
1,C18MB8E9FEO8O,2022-01-17,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X,N,1977.0,남자,현대닷컴,M5
2,C18MB8E9FEO8O,2022-01-17,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X,N,1977.0,남자,현대닷컴,M5
3,C18MB8E9FEO8O,2022-01-17,M5,포터II 디젤 슈퍼캡 초장축 카고 스타일 매뉴얼 22MY,소형상용,소형트럭,스타일,디젤,AGBCIS57DPHRRP9X,N,1977.0,남자,현대닷컴,M5
4,CXMMZDI2JRWCX,2020-05-11,DN,쏘나타DN8 20MY 자가용 가솔린 2.0 CVVL 프리미엄,승용,중형,프리미엄 패밀리,가솔린 2.0 CVVL,A4W2GG6ZLMZ81DB3,N,1974.0,여자,시승시스템,CN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126543,CF5XYYWGM4962,2011-10-25,MD,아반떼MD 자가용 가솔린 1.6 A 오토 11MY,승용,준중형,A,가솔린 1.6,ACWSOYPRRWLHL2I1,N,1979.0,남자,판매기회_상담완료,IG
126544,C91VRA1RAE6EL,2016-05-04,TL,투싼TL 디젤 1.7 Premium 2WD DCT,RV,준중형SUV,Premium,디젤 1.7,ACWSOYPRRWLHL2I1,N,1979.0,남자,판매기회_상담완료,IG
126545,COAFAA0KQ2F1N,2022-02-03,NX,The all-new 투싼 하이브리드 프리미엄 2WD A/T,RV,준중형SUV,프리미엄,하이브리드,AF3D2H4N6PO5PY24,N,1968.0,남자,판매기회_상담완료,NX
126546,COBZ4YAULMIMD,2010-07-29,JA,G스타렉스 디젤 12인승 웨건 자가용 CVX 오토 11MY,소형상용,소형버스,CVX,디젤,APXWMUKE36MD8PX1,N,1984.0,남자,판매기회_상담완료,WR


In [ ]:
df = df[['고객ID','생년','성별구분명','계약일자','차량ID','차종코드','승상구분명','채널구분명']]

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 126548 entries, 0 to 126547
Data columns (total 8 columns):
 #   Column  Non-Null Count   Dtype         
---  ------  --------------   -----         
 0   고객ID    126548 non-null  object        
 1   생년      126548 non-null  float64       
 2   성별구분명   126548 non-null  object        
 3   계약일자    126548 non-null  datetime64[ns]
 4   차량ID    126548 non-null  object        
 5   차종코드    126548 non-null  object        
 6   승상구분명   126548 non-null  object        
 7   채널구분명   126548 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 8.7+ MB


In [ ]:
## 분석할 연도 설정
df = df[df['계약일자'].between('2008-01-01', '2021-12-31')]

## 결측값 처리
df = df.dropna()

In [ ]:
df

,고객ID,생년,성별구분명,계약일자,차량ID,차종코드,승상구분명,채널구분명
0,APW9ZIWDQQNYY8LK,1992.0,남자,2021-03-02,C98N3HQIS4G67,IG,승용,시승시스템
4,A4W2GG6ZLMZ81DB3,1974.0,여자,2020-05-11,CXMMZDI2JRWCX,DN,승용,시승시스템
5,A4W2GG6ZLMZ81DB3,1974.0,여자,2020-05-11,CXMMZDI2JRWCX,DN,승용,시승시스템
22,A0G4BEZSV33U6T0B,1986.0,여자,2021-07-26,CA1M9D9IHFMYP,OS,RV,현대닷컴
23,AK4137SPJEPNX63F,1970.0,남자,2021-12-22,CGDV24KQXCAX0,NE,RV,시승시스템
...,...,...,...,...,...,...,...,...
126542,A7G45P5I4GJPNILT,1986.0,남자,2020-07-16,CN05IXLBICBB1,JS,승용,판매기회_상담완료
126543,ACWSOYPRRWLHL2I1,1979.0,남자,2011-10-25,CF5XYYWGM4962,MD,승용,판매기회_상담완료
126544,ACWSOYPRRWLHL2I1,1979.0,남자,2016-05-04,C91VRA1RAE6EL,TL,RV,판매기회_상담완료
126546,APXWMUKE36MD8PX1,1984.0,남자,2010-07-29,COBZ4YAULMIMD,JA,소형상용,판매기회_상담완료


In [ ]:
df = df.sort_values(by='계약일자')

In [ ]:
from tqdm import tqdm
from collections import Counter

## 먼저 각 고객별로 구매량이 얼마인가, 고객 고유치 : 51397
top_data = Counter(df['고객ID']).most_common() ## 데이터 개수가 많은 순으로 출력
monetary_df = pd.DataFrame() ## 구매량 데이터 초기화
# monetary_df['고객ID'] = top_data ## 고객아이디 삽입

data = [x[1] for x in top_data] ## 데이터 개수
cus = [x[0] for x in top_data] ## 고객

customer_id = list(cus)
monetary_df['고객ID'] = customer_id

monetary_data = [] ## 구매량을 담을 리스트
for i in tqdm(data, position=0, desc='Calculating amount of individual customer'):
    monetary_data.append(i) ## 해당 고객 구매량

monetary_df['Monetary'] = monetary_data ## 구매량 데이터 삽입
monetary_df['고객ID'] = cus

## 각 고객별 최근방문일을 알아보자.
temp_recency_df = df[['고객ID','계약일자']].drop_duplicates() ## 고객 아이디와 계약일자만 추출한뒤 중복 제거
recency_df = temp_recency_df.groupby('고객ID')['계약일자'].max().reset_index() ## 아이디로 그룹화 한다음 최근방문일을 구해야하므로 날짜에 max를 적용한다.
recency_df = recency_df.rename(columns={'계약일자':'Recency'})

## 각 고객별 방문횟수를 알아보자.
temp_frequency_df = df[['고객ID','계약일자']].drop_duplicates() ## 고객 아이디와 계약일자만 추출한뒤 중복 제거
frequency_df = temp_frequency_df.groupby('고객ID')['계약일자'].count().reset_index() ## 아이디로 그룹화 한다음 방문횟수를 구해야 한다. 여기서는 방문횟수를 계약일수로 생각했으므로 계약일수에 count를 적용한다.
frequency_df = frequency_df.rename(columns={'계약일자':'Frequency'})

## 데이터를 고객아이디를 기준으로 합쳐야한다.
rfm_df = pd.merge(recency_df,frequency_df,how='left',on='고객ID')
rfm_df = pd.merge(rfm_df,monetary_df,how='left',on='고객ID')

Calculating amount of individual customer: 100%|██████████| 32492/32492 [00:00<00:00, 1362567.99it/s]


In [ ]:
monetary_df.nunique()

고객ID        32492
Monetary       66
dtype: int64

In [ ]:
rfm_df.nunique()

고객ID         32492
Recency       3491
Frequency       19
Monetary        66
dtype: int64

In [ ]:
rfm_df

,고객ID,Recency,Frequency,Monetary
0,A0063H17IS0OELIO,2018-04-06,1,1
1,A007E35OUXTJ3BRR,2021-06-22,1,1
2,A008UGCQHSONWYDJ,2019-09-25,1,1
3,A00AXO66OKGQAC8F,2021-09-14,1,1
4,A00BD03LKJIH6LTU,2016-11-02,2,4
...,...,...,...,...
32487,AZZY0IZN75VOACLI,2018-12-03,1,1
32488,AZZYI8XJU7OCXO73,2011-06-27,1,1
32489,AZZYZ8SXW66M0RBL,2021-09-15,2,2
32490,AZZZ70FWYOY16G62,2011-02-18,3,6


In [ ]:
from sklearn import preprocessing

def get_score(level, data):

    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    return score
    
def get_rfm_grade(df, num_class, rfm_tick_point, rfm_col_map, suffix=None):
##### 최근방문일/방문횟수/구매금액 점수 부여
    for k, v in rfm_tick_point.items():
        if isinstance(v,str):
            if v == 'quantile':
                ## 데이터 변환
                scale = preprocessing.StandardScaler() ## 데이터의 범위 조작하기 쉽게 해주는 클래스 
                temp_data = np.array(df[rfm_col_map[k]]) ## 데이터를 Numpy 배열로 변환
                temp_data = temp_data.reshape((-1,1)) ## scale을 적용하기위해 1차원 배열을 2차원으로 변환
                temp_data = scale.fit_transform(temp_data) ## 데이터를 평균은 0, 표준편차는 1을 갖도록 변환 
                temp_data = temp_data.squeeze() ## 데이터를 다시 1차원으로 변환
 
                ## 분위수 벡터
                quantiles_level = np.linspace(0,1,num_class+1)[1:-1] ## 분위수를 구할 기준값을 지정 0과 1은 제외
                quantiles = [] ## 분위수를 담을 리스트
                for ql in quantiles_level:
                    quantiles.append(np.quantile(temp_data,ql)) ## 분위수를 계산하고 리스트에 삽입
            else: ## min_max인 경우
                ## 데이터 변환
                temp_data = np.array(df[rfm_col_map[k]])
 
                ## 등분점 계산
                quantiles = np.linspace(np.min(temp_data),np.max(temp_data),num_class+1)[1:-1] ## 최소값과 최대값을 점수 개수만큼 등간격으로 분할하는 점
        else: ## 직접 구분값을 넣어주는 경우
            temp_data = np.array(df[rfm_col_map[k]])
            quantiles = v ## 직접 구분값을 넣어줌
        score = get_score(quantiles, temp_data) ## 구분값을 기준으로 점수를 부여하고 리스트로 저장한다.
        new_col_name = rfm_col_map[k]+'_'+k ## 점수값을 담는 변수의 이름
        if suffix:
            new_col_name = rfm_col_map[k]+'_'+suffix
        df[new_col_name] = score ## 기존데이터 옆에 점수 데이터를 추가한다.
    return df

In [ ]:
##분위수를 이용해 점수 부여

rfm_tick_point={'R':'quantile','F':'quantile','M':'quantile'}
rfm_col_map={'R':'Recency','F':'Frequency','M':'Monetary'}

result = get_rfm_grade(df=rfm_df, num_class=5, rfm_tick_point=rfm_tick_point, rfm_col_map=rfm_col_map)
result.to_csv('result.csv',index=False)

In [ ]:
result[['고객ID','Recency_R','Frequency_F','Monetary_M']]

,고객ID,Recency_R,Frequency_F,Monetary_M
0,A0063H17IS0OELIO,3,1,1
1,A007E35OUXTJ3BRR,5,1,1
2,A008UGCQHSONWYDJ,3,1,1
3,A00AXO66OKGQAC8F,5,1,1
4,A00BD03LKJIH6LTU,2,4,5
...,...,...,...,...
32487,AZZY0IZN75VOACLI,3,1,1
32488,AZZYI8XJU7OCXO73,1,1,1
32489,AZZYZ8SXW66M0RBL,5,4,3
32490,AZZZ70FWYOY16G62,1,5,5


### ***rfm_score data 설명***

1. Recency = 최근 구매일
2. Frequency = 방문횟수
3. Monetary = 구매횟수
4. Recency_R = 최근 구매일 점수, 1~5점으로 점수가 높을수록 좋다.
5. Frequency_F = 방문횟수 점수, 1~5점으로 점수가 높을수록 좋다.
6. Monetary_M = 구매횟수 점수, 1~5점으로 점수가 높을수록 좋다.

In [ ]:
## data load
rfm_score = pd.read_csv('result.csv')
rfm_score.head(10)

,고객ID,Recency,Frequency,Monetary,Recency_R,Frequency_F,Monetary_M
0,A0063H17IS0OELIO,2018-04-06,1,1,3,1,1
1,A007E35OUXTJ3BRR,2021-06-22,1,1,5,1,1
2,A008UGCQHSONWYDJ,2019-09-25,1,1,3,1,1
3,A00AXO66OKGQAC8F,2021-09-14,1,1,5,1,1
4,A00BD03LKJIH6LTU,2016-11-02,2,4,2,4,5
5,A00BKTGNMZXXU9NB,2015-09-25,1,1,2,1,1
6,A00E949YXDIP24UM,2021-12-30,1,1,5,1,1
7,A00EIFH5MF1D3N68,2010-04-19,1,3,1,1,4
8,A00EKMHO59OQNRCX,2016-12-16,1,2,2,1,3
9,A00FF7U0QU7LK2E5,2012-02-20,2,2,1,4,3


In [ ]:
rfm_score = rfm_score[['고객ID', 'Monetary', 'Recency_R','Frequency_F','Monetary_M']]

In [ ]:
def get_score(level, data, reverse = False):
    score = [] 
    for j in range(len(data)): 
        for i in range(len(level)): 
            if data[j] <= level[i]: 
                score.append(i+1) 
                break 
            elif data[j] > max(level): 
                score.append(len(level)+1) 
                break 
            else: 
                continue
    if reverse:
        return [len(level)+2-x for x in score]
    else:
        return score 
 
grid_number = 100 ## 눈금 개수, 너무 크게 잡으면 메모리 문제가 발생할 수 있음.
weights = []
for j in range(grid_number+1):
    weights += [(i/grid_number,j/grid_number,(grid_number-i-j)/grid_number)
                  for i in range(grid_number+1-j)]
num_class = 5 ## 클래스 개수
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
total_amount_of_sales = rfm_score['Monetary'].sum() ## 구매금액 총합 = 총 매출

In [ ]:
# class의 구간치, 1.8 이하면 5등급, 4.2이상이면 1등급 고객

num_class = 5
class_level = np.linspace(1,5,num_class+1)[1:-1] ## 클래스를 나누는 지점을 정한다.
print(class_level)

[1.8 2.6 3.4 4.2]


In [ ]:
## 최적의 표준편차 값을 구하는 Logic

max_std = 0 ## 표준편차 초기값

for w in tqdm(weights,position=0,desc = '[Finding Optimal weights]'):
    ## 주어진 가중치에 따른 고객별 점수 계산
    score = w[0]*rfm_score['Recency_R'] + \
                        w[1]*rfm_score['Frequency_F'] + \
                        w[2]*rfm_score['Monetary_M'] 
    rfm_score['Class'] = get_score(class_level,score,True) ## 점수를 이용하여 고객별 등급 부여
    ## 등급별로 구매금액을 집계한다.
    grouped_rfm_score = rfm_score.groupby('Class')['Monetary'].sum().reset_index()

    ## 등급이 높은 고객들의 매출이 낮은 등급의 고객들보다 커야한다.
    grouped_rfm_score = grouped_rfm_score.sort_values('Class')
    temp_monetary = list(grouped_rfm_score['Monetary'])
    if temp_monetary != sorted(temp_monetary,reverse=True):
        continue
        
    ## 클래스별 구매금액을 총구매금액으로 나누어 클래스별 매출 기여도 계산
    grouped_rfm_score['Monetary'] = grouped_rfm_score['Monetary'].map(lambda x : x/total_amount_of_sales)
    std_sales = grouped_rfm_score['Monetary'].std() ## 매출 기여도의 표준편차 계산

    ## 표준편차 최대값 업데이트 및 가중치 업데이트
    if max_std <= std_sales:
        max_std = std_sales 
        optimal_weights = w

[Finding Optimal weights]: 100%|██████████| 5151/5151 [33:34<00:00,  2.56it/s]


In [ ]:
print('Recency_R, Frequency_F, Monetary_M의 최적의 가중치 :',optimal_weights)

Recency_R, Frequency_F, Monetary_M의 최적의 가중치 : (0.36, 0.04, 0.6)


In [ ]:
score = optimal_weights[0]*rfm_score['Recency_R'] + \
        optimal_weights[1]*rfm_score['Frequency_F'] + \
        optimal_weights[2]*rfm_score['Monetary_M'] ## 고객별 점수 계산

rfm_score['Class'] = get_score(class_level,score,True) ## 고객별 등급 부여

In [ ]:
rfm_score

,고객ID,Monetary,Recency_R,Frequency_F,Monetary_M,Class
0,A0063H17IS0OELIO,1,3,1,1,5
1,A007E35OUXTJ3BRR,1,5,1,1,4
2,A008UGCQHSONWYDJ,1,3,1,1,5
3,A00AXO66OKGQAC8F,1,5,1,1,4
4,A00BD03LKJIH6LTU,4,2,4,5,2
...,...,...,...,...,...,...
32487,AZZY0IZN75VOACLI,1,3,1,1,5
32488,AZZYI8XJU7OCXO73,1,1,1,1,5
32489,AZZYZ8SXW66M0RBL,2,5,4,3,2
32490,AZZZ70FWYOY16G62,6,1,5,5,2


In [ ]:
## 클래스별 고객 수 계산
temp_rfm_score1 = rfm_score.groupby('Class')['고객ID'].count().reset_index().rename(columns={'CustomerID':'Count'})
 
## 클래스별 구매금액(매출)계산
temp_rfm_score2 = rfm_score.groupby('Class')['Monetary'].sum().reset_index()

## 클래스별 매출 기여도 계산
temp_rfm_score2['Contribution'] = temp_rfm_score2['Monetary'].map(lambda x : x/total_amount_of_sales)
 
## 데이터 결합
result_df = pd.merge(temp_rfm_score1,temp_rfm_score2,how='left',on=('Class'))

In [ ]:
result_df

,Class,고객ID,Monetary
0,1,3545,0.347959
1,2,3894,0.186004
2,3,5488,0.173192
3,4,9184,0.157685
4,5,10381,0.135160
